In [1]:
# !unzip dataset_strw_vs_pear.zip

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
# import idx2numpy
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
from PIL import Image
from tqdm.notebook import tqdm
from torch.utils.tensorboard import SummaryWriter
import tempfile
import torchvision.models as models
import torchvision
from torch.optim import lr_scheduler
import time
import copy
from google.colab import drive
import matplotlib.image as mpimg
from PIL import Image
drive.mount("/content/gdrive")
import os
from google.colab import files
import shutil

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
class VGG_1b(nn.Module):
    def __init__(self):
        super(VGG_1b, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.fc1 = nn.Linear(320000, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 2)

    def forward(self, x):
        # print('hi')
        # print(x.shape)
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.dropout(x, p=0.5, training=self.training)
        x = nn.functional.relu(self.fc2(x))
        x = nn.functional.dropout(x, p=0.5, training=self.training)
        x = self.fc3(x)
        return x


In [4]:
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=3)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=3)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=3)
        self.fc1 = nn.Linear(100352, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 2)

    def forward(self, x):

        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.conv3(x))
        x = nn.functional.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.dropout(x, p=0.5, training=self.training)
        x = nn.functional.relu(self.fc2(x))
        x = nn.functional.dropout(x, p=0.5, training=self.training)
        x = self.fc3(x)
        return x


In [5]:
class FeedForwardNetwork(nn.Module):
    """layerwise_hidden_dims is a list of hidden dimension size in consecutive layers. layerwise_activations is a list that stores the activation function to be used after each layer."""
    def __init__(self, input_size, output_size, layerwise_hidden_dims, layerwise_activations):
        super(FeedForwardNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, layerwise_hidden_dims[0]),
            layerwise_activations(),
            # nn.Linear(layerwise_hidden_dims[0], layerwise_hidden_dims[1]),
            # layerwise_activations(),
            nn.Linear(layerwise_hidden_dims[0], output_size)
        )

    def forward(self, x):
        x = self.flatten(x)
#         print(x, x.shape)
#         x = x.view(-1, 3*28*28)
        logits = self.linear_relu_stack(x)
        return logits

In [6]:
def train(train_dataloader, test_dataloader, model, criterion, optimizer, device, writer, epochs=10, model_str="VGG3"):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    no_of_steps = 0
    no_of_batches = len(train_dataloader)
    size = len(train_dataloader.dataset)
    model.train()
    for epoch in tqdm(range(epochs), desc="Epochs"):
        running_loss, total_loss, correct = 0.0, 0.0, 0
        for i, data in enumerate(train_dataloader, 0):
            inputs, labels = data[0].to(device), data[1].type(torch.LongTensor).to(device)
            optimizer.zero_grad()
            
            outputs = model(inputs.cuda())

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            cur_loss = loss.item()
            running_loss += cur_loss
            total_loss += cur_loss
            no_of_steps += 1

            correct += (outputs.argmax(1) == labels).type(torch.float).sum().item()
            writer.add_scalar(f'Training Loss VS Iteration', loss, no_of_steps)
            
            ##########Train Accuracy for each iteration###########
            model.eval()
            loc_correct = 0
            with torch.no_grad():
                for j, data_ in enumerate(train_dataloader, 0):
                    inputs_, labels_ = data_[0].to(device), data_[1].type(torch.LongTensor).to(device)
                    pred_ = model(inputs_)
                    loc_correct += (pred_.argmax(1) == labels_).type(torch.float).sum().item()

            loc_correct /= size
            writer.add_scalar('Training Accuracy VS Iteration', 100*loc_correct, no_of_steps)

            ##########Train Accuracy for each iteration###########
            # model.eval()
            loc_correct = 0
            with torch.no_grad():
                for j, data_ in enumerate(test_dataloader, 0):
                    inputs_, labels_ = data_[0].to(device), data_[1].type(torch.LongTensor).to(device)
                    pred_ = model(inputs_)
                    loc_correct += (pred_.argmax(1) == labels_).type(torch.float).sum().item()

            loc_correct /= size
            writer.add_scalar('Testing Accuracy VS Iteration', 100*loc_correct, no_of_steps)

            model.train()            


        total_loss /= no_of_batches
        correct /= size
        writer.add_scalar('Training Accuracy VS Epoch', 100*correct, epoch)
        writer.add_scalar('Avg Training Loss VS Epoch', total_loss, epoch) 
        print(f'Epoch:{epoch} | Training Accuracy:{100*correct} | Training Loss:{total_loss}')

        model.eval()
        loc_correct = 0
        with torch.no_grad():
            for j, data_ in enumerate(test_dataloader, 0):
                inputs_, labels_ = data_[0].to(device), data_[1].type(torch.LongTensor).to(device)
                pred_ = model(inputs_)
                loc_correct += (pred_.argmax(1) == labels_).type(torch.float).sum().item()

        loc_correct /= size
        writer.add_scalar('Test Accuracy VS Epoch', 100*loc_correct, no_of_steps)
        model.train() 
    
    print()
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    # model.load_state_dict(best_model_wts)
    return model



In [7]:
def test(test_dataloader, model, loss_fn, device, writer, model_str="VGG3"):

    size = len(test_dataloader.dataset)
    num_batches = len(test_dataloader)
    model.eval()
    test_loss, correct = 0, 0
    count = 0
    with torch.no_grad():
        for i, data in enumerate(test_dataloader, 0):
            inputs, labels = data[0].to(device), data[1].type(torch.LongTensor).to(device)
#             X = torch.mean(X, 1)
            pred = model(inputs)
            test_loss += loss_fn(pred, labels).item()
            correct += (pred.argmax(1) == labels).type(torch.float).sum().item()
            # writer.add_scalar(f'Test Accuracy VS Step', correct, epoch*(num_batches)+count)
            count += 1 
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {100*(correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
# Write the custom Dataset class here
class CustomImageDataset(Dataset):
    # image_path = '/content/train-images-idx3-ubyte' label_path='/content/train-labels-idx1-ubyte'
    def __init__(self, image_path, label_path, transform=None):

        self.images = np.load(image_path)
        self.labels = np.load(label_path)
        self.transform = transform

    def __len__(self):
        return self.images.shape[0]

    def __getitem__(self, idx):

        image_array = self.images[idx]
        label = self.labels[idx]
        image_array = image_array[:3]
        return image_array, label

In [9]:
def get_tranforms(data_aug):
    
    train_transforms = transforms.Compose([
        transforms.Resize(200),
        transforms.CenterCrop(200),
        transforms.ToTensor(),
        # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

    test_transforms = transforms.Compose([
        transforms.Resize(200),
        transforms.CenterCrop(200),
        transforms.ToTensor(),
        # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
    # data_aug = False

    # if data_aug:
    #     train_transforms = transforms.Compose([transforms.RandomRotation(30),
    #                                        transforms.RandomResizedCrop(200),
    #                                        transforms.RandomHorizontalFlip(),
    #                                        transforms.ToTensor(),
    #                                        transforms.Normalize([0.5, 0.5, 0.5], 
    #                                                             [0.5, 0.5, 0.5])])
    return train_transforms, test_transforms

In [10]:
# training_data = CustomImageDataset(
#     'train_data.npy', 'train_label.npy', transform)
# testing_data = CustomImageDataset('test_data.npy', 'test_label.npy', transform)
def get_loader(model_str):
  data_aug = False
  train_transforms, test_transforms = get_tranforms(data_aug)
  data_dir = '/content/gdrive/My Drive/dataset_strw_vs_pear'
  if model_str =='VGG3_aug':
    data_dir = '/content/gdrive/My Drive/dataset_strw_vs_pear_aug'
    print("Augmented Dataset Taken")

  training_data = datasets.ImageFolder(data_dir + '/train',  
                                      transform=train_transforms)                                       
  testing_data = datasets.ImageFolder(data_dir + '/test', 
                                      transform=test_transforms)

  train_dataloader = DataLoader(training_data, batch_size=32, shuffle=True)
  test_dataloader = DataLoader(testing_data, batch_size=32, shuffle=True)

  return train_dataloader, test_dataloader

In [11]:

def model_init(model_str, writer):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model = VGG().to(device)

  input_size = 120000#200 * 200 *3
  output_size = 2
  layerwise_hidden_dims = [1119]# [25*25, 10*5*2]
  layerwise_activations = nn.ReLU

  # model_str = "VGG3"
  criterion = nn.CrossEntropyLoss()
  if model_str == "VGG3":
    model = VGG().to(device)
  elif model_str == "VGG1":
    model = VGG_1b().to(device)
  elif model_str == "VGG3_aug":
    model = VGG().to(device)
    pass
  elif model_str == "VGG16":
    print("Inside VGG16")
    model = models.vgg16(pretrained=True).to(device)
    mod = list(model.classifier.children())
    mod.pop()
    mod.append(torch.nn.Linear(4096, 2))
    new_classifier = torch.nn.Sequential(*mod)
    model.classifier = new_classifier.cuda()
  elif model_str == "MLP":
    print("Inside MLP")
    model = FeedForwardNetwork(input_size, output_size, 
                              layerwise_hidden_dims, layerwise_activations).to(device)
  
  optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

  train_dataloader, test_dataloader = get_loader(model_str)
  
  model_trained = train(train_dataloader, test_dataloader, model, criterion, optimizer, device, writer, epochs=12, model_str=model_str)
  PATH = "./saved_models"
  # Create directory if it doesn't exist
  if not os.path.exists(PATH):
      os.makedirs(PATH)
  # Save the model to the directory
  save_path = os.path.join(PATH, f"model_{model_str}.pt")
  torch.save(model.state_dict(), save_path)
  # Zip the folder you want to download
  shutil.make_archive('saved_models', 'zip', '/content/saved_models')

  # Download the zip file
  files.download('saved_models.zip')
  test(test_dataloader, model, criterion, device, writer, model_str=model_str)
  return model

In [12]:
%load_ext tensorboard
!rm -rf ./logs/
# log_dir = tempfile.mkdtemp()   # requires a temp dir tpo store logs
%tensorboard --logdir runs  --reload_interval 1  ##reload log every 1 sec

<IPython.core.display.Javascript object>

In [13]:
%reload_ext tensorboard
m = "MLP"
model_name = f'/train/model_{m}'
writer = SummaryWriter(log_dir='runs/{}'.format("tensorboard") + model_name)

In [ ]:
model_trained = model_init(m, writer)

Inside MLP


Epochs:   0%|          | 0/12 [00:00<?, ?it/s]

Epoch:0 | Training Accuracy:67.5 | Training Loss:0.5259327590465546
Epoch:1 | Training Accuracy:92.5 | Training Loss:0.21874331831932067
Epoch:2 | Training Accuracy:95.0 | Training Loss:0.14481711238622666
Epoch:3 | Training Accuracy:97.5 | Training Loss:0.06974354535341262
Epoch:4 | Training Accuracy:100.0 | Training Loss:0.041161917895078656
Epoch:5 | Training Accuracy:100.0 | Training Loss:0.0238040030002594
Epoch:6 | Training Accuracy:100.0 | Training Loss:0.01978818476200104
Epoch:7 | Training Accuracy:100.0 | Training Loss:0.013952456414699554


In [ ]:
# testing_data.images.shape, testing_data.labels.shape, training_data.images.shape, training_data.labels.shape

In [ ]:
train_dataloader, test_dataloader = get_loader(m)


In [ ]:

test_dataloader.dataset.classes

In [ ]:
device = "cuda"
true_labels = []
pred_labels = []
image_list = []
size = len(test_dataloader.dataset)
num_batches = len(test_dataloader)
model_trained.eval()
test_loss, correct = 0, 0
count = 0
with torch.no_grad():
    for i, data in enumerate(test_dataloader, 0):
        inputs, labels = data[0].to(device), data[1].type(torch.LongTensor).to(device)
#             X = torch.mean(X, 1)
        pred = model_trained(inputs)
        # test_loss += loss_fn(pred, labels).item()
        correct += (pred.argmax(1) == labels).type(torch.float).sum().item()
        print(f'Label type:{type(labels)}, Pred argmax type: {type(pred.argmax(1))}, Input type:{type(inputs)}')
        for i in range(len(labels)):
          true_labels.append(labels[i])
          pred_labels.append(pred.argmax(1)[i])
          image_list.append(inputs[i])
          # print(labels)


In [ ]:
# true_labels[0].item()
# pred_labels[0].item()
# image_list[0].shape
print(len(image_list))
import matplotlib.image as mpimg
from PIL import Image
type(pred_labels[0].item())

In [ ]:
pred_labels[0]

In [ ]:

import io

In [ ]:
fig, axs = plt.subplots(8, 5, figsize=(20, 20))
fig.suptitle(f'{m} model predictions')
# img = transform(tensor)
count = 0
img_tranform = transforms.ToPILImage()
for i in range(8):
  for j in range(5):
    axs[i, j].imshow(img_tranform(image_list[count]))
    pred_str = "Pear" if pred_labels[count].item() == 0 else "Strawberry"
    axs[i, j].set_title(f'Predicted:{pred_str}')
    axs[i, j].axis('off')
    count += 1

# buf = io.BytesIO()
# fig.savefig(buf, format='jpeg')
# buf.seek(0)



In [ ]:
PATH = "./gdrive/MyDrive/ML_AS5_models/saved_models"
  # Create directory if it doesn't exist
if not os.path.exists(PATH):
    os.makedirs(PATH)
# Save the model to the directory
save_path = os.path.join(PATH, f"model_{m}.pt")
torch.save(model_trained.state_dict(), save_path)
# Zip the folder you want to download
shutil.make_archive('saved_models', 'zip', PATH)

# Download the zip file
files.download('saved_models.zip')

In [ ]:

writer.add_figure(f'{m} model predicted image', fig)

In [ ]:

def count_parameters(model):
    """
    Counts the total number of trainable parameters in a PyTorch model.
    """
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


In [ ]:
print(f"Total Parameters of model {m} are: {count_parameters(model_trained)}")

In [ ]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

count_parameters(model_trained)

In [ ]:
# Zip the folder you want to download
import shutil
shutil.make_archive('saved_models', 'zip', '/content/saved_models')

# Download the zip file
from google.colab import files
files.download('saved_models.zip')


In [ ]:
import shutil
shutil.make_archive('runs', 'zip', '/content/runs')

# Download the zip file
from google.colab import files
files.download('runs.zip')

In [ ]:
134268738

150126552
75063427



| |Training time | Training loss | Training accuracy | Testing accuracy | No of model parameters|
|-|--- | --- | --- | --- | --- |
|VGG (1 block)| 21m 42s |  0.048715802282094954 | 99.375 | 95.0% | 163907330 |
|VGG (3 blocks)| 22m 11s | 0.2673647880554199  | 90.625 | 95.0% | 103117634 |
|VGG (3 blocks) with data augmentation | 44m 40s |  0.16372035369277 | 93.75 | 100.0% | 103117634 |
|Transfer learning using VGG16| 13m 19s(only 12 epochs) | 0.00017252258185180837 | 100 | 100.0% | 134268738 |
|MLP|  |   |  |  | 12002677 |


25 epochs


+--------------+------------+
|   Modules    | Parameters |
+--------------+------------+
| conv1.weight |    864     |
|  conv1.bias  |     32     |
| conv2.weight |   18432    |
|  conv2.bias  |     64     |
| conv3.weight |   73728    |
|  conv3.bias  |    128     |
|  fc1.weight  | 102760448  |
|   fc1.bias   |    1024    |
|  fc2.weight  |   262144   |
|   fc2.bias   |    256     |
|  fc3.weight  |    512     |
|   fc3.bias   |     2      |
+--------------+------------+
Total Trainable Params: 103117634
VGG3

103117634

VGG1

+--------------+------------+
|   Modules    | Parameters |
+--------------+------------+
| conv1.weight |    864     |
|  conv1.bias  |     32     |
|  fc1.weight  | 163840000  |
|   fc1.bias   |    512     |
|  fc2.weight  |   65536    |
|   fc2.bias   |    128     |
|  fc3.weight  |    256     |
|   fc3.bias   |     2      |
+--------------+------------+
Total Trainable Params: 163907330

163907330

VGG3_aug

+--------------+------------+
|   Modules    | Parameters |
+--------------+------------+
| conv1.weight |    864     |
|  conv1.bias  |     32     |
| conv2.weight |   18432    |
|  conv2.bias  |     64     |
| conv3.weight |   73728    |
|  conv3.bias  |    128     |
|  fc1.weight  | 102760448  |
|   fc1.bias   |    1024    |
|  fc2.weight  |   262144   |
|   fc2.bias   |    256     |
|  fc3.weight  |    512     |
|   fc3.bias   |     2      |
+--------------+------------+
Total Trainable Params: 103117634

103117634

Testing Loss: 0.093954 

VGG16

+---------------------+------------+
|       Modules       | Parameters |
+---------------------+------------+
|  features.0.weight  |    1728    |
|   features.0.bias   |     64     |
|  features.2.weight  |   36864    |
|   features.2.bias   |     64     |
|  features.5.weight  |   73728    |
|   features.5.bias   |    128     |
|  features.7.weight  |   147456   |
|   features.7.bias   |    128     |
|  features.10.weight |   294912   |
|   features.10.bias  |    256     |
|  features.12.weight |   589824   |
|   features.12.bias  |    256     |
|  features.14.weight |   589824   |
|   features.14.bias  |    256     |
|  features.17.weight |  1179648   |
|   features.17.bias  |    512     |
|  features.19.weight |  2359296   |
|   features.19.bias  |    512     |
|  features.21.weight |  2359296   |
|   features.21.bias  |    512     |
|  features.24.weight |  2359296   |
|   features.24.bias  |    512     |
|  features.26.weight |  2359296   |
|   features.26.bias  |    512     |
|  features.28.weight |  2359296   |
|   features.28.bias  |    512     |
| classifier.0.weight | 102760448  |
|  classifier.0.bias  |    4096    |
| classifier.3.weight |  16777216  |
|  classifier.3.bias  |    4096    |
| classifier.6.weight |    8192    |
|  classifier.6.bias  |     2      |
+---------------------+------------+
Total Trainable Params: 134268738

134268738


Test Error: 
 Accuracy: 100.0%, Avg loss: 0.000079 

MLP


+----------------------------+------------+
|          Modules           | Parameters |
+----------------------------+------------+
| linear_relu_stack.0.weight |  12000000  |
|  linear_relu_stack.0.bias  |    100     |
| linear_relu_stack.2.weight |    2500    |
|  linear_relu_stack.2.bias  |     25     |
| linear_relu_stack.4.weight |     50     |
|  linear_relu_stack.4.bias  |     2      |
+----------------------------+------------+
Total Trainable Params: 12002677

12002677